In [2]:
from py_vlasov.util import zp, pade, zp_mp, VlasovException, real_imag
from py_vlasov.util import (pmass, emass, echarge, permittivity, permeability, cspeed, boltzmann)
import numpy as np
import scipy.optimize

In [ ]:
def f_zeta(w, kz, vz, Omega, vthz, n):
    """
    Calculate the argument of plasma dispersion function.
    
    Keyword arguments
    -----------------
    w: frequency (rad/s)
    kz: parallel wavenumber (rad/m)
    vz: parallel drift of the particle species (m/s)
    Omega: gyrofrequency of the species (rad/s)
    vthz: parallel thermal speed (m/s)
    n: resonance number
    
    Return
    ------
    \zeta_{ns}
    """
    return (w-kz*vz-n*Omega)/(kz*vthz)

In [ ]:
def choose_zp_fn(method):
    """
    choose which function to calclate the plasma dispersion function.
    
    Keyword arguments
    -----------------
    method: a string in ['pade', 'numpy', 'mpmath']
    
    Return
    ------
    return the pointer to the function object.
      
    """
    if method == 'pade':
        f_zp = pade
    elif method == 'numpy':
        f_zp = zp
    elif method == 'mpmath':
        f_zp = zp_mp
    else:
        raise VlasovException("Unreconized method.\n" +
            "Please choose between 'pade', 'numpy' and 'mpmath'")
    return f_zp

In [ ]:
def r_wave_rhs(n, w, kz, kp, wp, tz, tp, vthz, vthp, Omega, vz, method = 'pade'):
    """

    Keyword arguments
    -----------------
    n: number of terms to sum over. do not need for parallel propagation.
    w: frequency
    kz: parallel wavenumber
    kp: perpendicular wavenumber. kp = 0 for parallel propagation.
    wp: plasma frequency of the species
    tz: parallel temperature
    tp: perpendicular temperature
    vthz: parallel thermal speed
    vthp: perpendicular thermal speed
    Omega: gyrofrequency
    vz: parallel drift
    
    Return
    ------
    The value of the summed term on the RHS of Eq (2), P. 267, Stix (1992). 
    Eq (2) yields R wave.
    """
    term_1 = -(tp-tz)/tz
    term_2 = ((w - kz*vz + Omega)*tp - Omega*tz)/(kz * vthz * tz)
    f_zp = choose_zp_fn(method)
    zeta = f_zeta(w, kz, vz, Omega, vthz, -1)
    term_3 = f_zp(zeta)
    #print("zeta", zeta, "Z", term_3)
    rhs = wp**2 * (term_1 + term_2 * term_3)
    return rhs

In [ ]:
def l_wave_rhs(n, w, kz, kp, wp, tz, tp, vthz, vthp, Omega, vz, method = 'pade'):
    """

    Keyword arguments
    -----------------
    n: number of terms to sum over. do not need for parallel propagation.
    w: frequency
    kz: parallel wavenumber
    kp: perpendicular wavenumber. kp = 0 for parallel propagation.
    wp: plasma frequency of the species
    tz: parallel temperature
    tp: perpendicular temperature
    vthz: parallel thermal speed
    vthp: perpendicular thermal speed
    Omega: gyrofrequency
    vz: parallel drift
    
    Return
    ------
    The value of an the summed term in Eq (3), P. 267, Stix (1992). 
    Eq (3) yields L wave.
    """
    term_1 = (tp-tz)/tz
    term_2 = ((w - kz*vz - Omega)*tp + Omega*tz)/(kz * vthz * tz)
    f_zp = choose_zp_fn(method)
    zeta = f_zeta(w, kz, vz, Omega, vthz, 1)
    term_3 = f_zp(zeta)
    rhs = wp**2 * (term_1 + term_2 * term_3)
    return rhs

In [ ]:
def static_rhs(n, w, kz, kp, wp, tz, tp, vthz, vthp, Omega, vz, method = 'pade'):
    """

    Keyword arguments
    -----------------
    n: number of terms to sum over. do not need for parallel propagation.
    w: frequency
    kz: parallel wavenumber
    kp: perpendicular wavenumber. kp = 0 for parallel propagation.
    wp: plasma frequency of the species
    tz: parallel temperature
    tp: perpendicular temperature
    vthz: parallel thermal speed
    vthp: perpendicular thermal speed
    Omega: gyrofrequency
    vz: parallel drift
    
    Return
    ------
    The value of an the summed term in Eq (4), P. 267, Stix (1992). 
    Eq (4) yields electrostatic wave.
    """
    term_1 = 2 * (wp/ kz / vthz)**2
    term_2 = (w - kz * vz)/ (kz * vthz)
    f_zp = choose_zp_fn(method)
    zeta = f_zeta(w, kz, vz, Omega, vthz, 0)
    term_3 = f_zp(zeta)
    rhs = term_1 * (1 + term_2 * term_3)
    return rhs

In [ ]:
def r_wave_eqn(param):
    """
    Keyword arguments
    -----------------
    param: a 2D list, where param[:, j] = [n_j, w, kz, kp, wp_j, tz_j, tp_j, vthz_j, vthp_j, Omega_j, vz_j, method = 'pade']
    
    Return
    ------
    Return the value of dispersion equation for R wave.    
    """
    w = param[1][0]
    kz = param[2][0]
    return w**2 + np.sum(np.array(list(map(r_wave_rhs, *param))), axis = 0) - (kz * cspeed)**2

In [ ]:
def l_wave_eqn(param):
    """
    Keyword arguments
    -----------------
    param: a 2D list, where param[:, j] = [n_j, w, kz, kp, wp_j, tz_j, tp_j, vthz_j, vthp_j, Omega_j, vz_j, method = 'pade']
    
    Return
    ------
    Return the value of dispersion equation for L wave.    
    """
    w = param[1][0]
    kz = param[2][0]
    return w**2 + np.sum(np.array(list(map(l_wave_rhs, *param))), axis = 0) - (kz * cspeed)**2

In [ ]:
def static_wave_eqn(param):
    """
    Keyword arguments
    -----------------
    param: a 2D list, where param[:, j] = [n_j, w, kz, kp, wp_j, tz_j, tp_j, vthz_j, vthp_j, Omega_j, vz_j, method = 'pade']
    
    Return
    ------
    Return the value of dispersion equation for electrostatic waves.    
    """
    return 1 + np.sum(np.array(list(map(static_rhs, *param))), axis = 0)

In [ ]:
def parallel_em_wave_wrapper(wrel, k, betap, tep, 
                             ap, ae, method = 'pade', 
                             mratio=1836, aol=1/5000):
    """
    Consider a hydrogen plasma. Takes in dimensionless arguments \
    and return value of the dispersion equation of parallel-progating \
    EM waves.
    
    Keyword arguments
    -----------------
    tep: T_{e\parallel}/T_{p\parallel}
    ap: ap\equiv 1 - T_{p\perp}/T_{p\parallel}
    ae: ae\equiv 1 - T_{e\perp}/T_{e\parallel}
    
    Return
    ------
    Return the value of dispersion equation for R wave. 
    Eq (2), P. 267, Stix(1992)
    """
    if mratio == 1836:
        emass_local = emass
    else:
        emass_local = pmass/mratio
        
    # by default add over 10 terms
    b0 = 1e-8      # 10nT by default
    vz = 0         # no bulk drift
    va = cspeed * aol
    nproton = (b0/va)**2 / (permeability * pmass)
    # T_{p\parallel}
    tp = betap * b0**2 / (2 * permeability * nproton * boltzmann)
    tp_perp = tp * (1 - ap)
    te = tp * tetp
    te_perp = te * (1 - ae)
    wpp = np.sqrt(nproton * echarge**2 / (pmass * permittivity))
    wpe = np.sqrt(nproton * echarge**2 / (emass_local * permittivity))
    omega_p = echarge * b0/ pmass # proton gyro-freqeuncy
    omega_e = -echarge * b0/emass_local
    vthp = np.sqrt(2 * boltzmann * tp/pmass) # proton parallel thermal speed
    vthe = np.sqrt(2 * boltzmann * te/emass_local) # electron parallel thermal speed
    rhop = vthp/omega_p # proton gyroradius (parallel)
    w = wrel * omega_p
    kz = k/rhop
    kp = 0
    vthp_perp = vthp * np.sqrt(1 - ap) 
    vthe_perp = vthe * np.sqrt(1 - ae)
    
    proton = [n, w, kz, kp, wpp, tp, tp_perp, vthp, vthp_perp, omega_p, 0, method]
    electron = [n, w, kz, kp, wpe, te, te_perp, vthe, vthe_perp, omega_e, 0, method]
    
    inp = [proton, electron]
    param = list(map(list, zip(*inp)))
    return r_wave_eqn(param)

In [ ]:
def parallel_em_wave_wrapper_1(wrel, k, betap, t_list, a_list, n_list, q_list,
                             m_list, v_list, method = 'pade', aol=1/5000, mode = 'r'):
    """
    A more systematic way to consider multiple component plasmas.
    Assume that THE FIRST COMPONENT IS ALWAYS PROTON.
    
    Kyeword arguments
    -----------------
    wrel: dimensionless wave frequency 
        \omega/\Omega_p
    k: dimensionless wave number
        k * \rho_{p\parallel}
    betap: proton parallel beta
        \beta_{p\parallel}, 
    t_list: temperature ratio T_{s\parallel}/T_{p\parallel}.
        where s --> species. The first component by default represent proton.
    a_list: temperature anisotropy
        a_s \equiv 1 - T_{s\perp}/T_{s\parallel}
    n_list: density fraction
        n_s \equiv n_s/n_p, n_p --> proton density
    q_list: charge in unit of proton charge.
    m_list: mass ratio
        m_s \equiv m_s/m_p, m_p --> proton mass.
    v_list: dimensionless bulk drift.
        v_{ds} = v_{ds}/v_A, where v_A --> Alfven speed
    
    """
    b0 = 1e-8 # 10nT by default
    va = cspeed * aol # Alfven speed
    nproton = (b0/va)**2 / (permeability * pmass)
    tp_par = betap * b0**2 / (2 * permeability * nproton * boltzmann)
    omega_p = echarge * b0/pmass # proton gyrofrequency
    vthp_par = np.sqrt(2 * boltzmann * tp_par/pmass) # proton parallel thermal speed
    rhop_par = vthp_par/omega_p
    w = wrel * omega_p
    kz = k/rhop_par
    kp = 0 # parallel propogation.
    n = 0 # no summation for parallel modes. Unnecessary parameters
    inp = []
    
    for i in range(len(t_list)):
        ns = nproton * n_list[i]
        ts_par = tp_par * t_list[i]
        ts_perp = ts_par * (1 - a_list[i])
        ms = pmass * m_list[i]
        vds = va * v_list[i]
        qs = echarge * q_list[i]
        wps = np.sqrt(ns * qs**2 / (ms * permittivity))
        omegas = qs * b0/ms
        vths_par = np.sqrt(2 * boltzmann * ts_par/ms) 
        vths_perp = np.sqrt(2 * boltzmann * ts_perp/ms)
        species = [n, w, kz, kp, wps, ts_par, ts_perp, vths_par, vths_perp, omegas, vds, method]
        inp += [species]
    param = list(map(list, zip(*inp)))
    if mode == 'r':
        res = r_wave_eqn(param) 
    elif mode == 'l':
        res = l_wave_eqn(param) 
    elif mode == 's':
        res = static_eqn(param)
    #res = np.real(res) + 1e70j * np.imag(res)
    print(res)
    return res    

## Below we start to test against whamp

##### Whamp input file

In [ ]:
def fce(B):
    """
    Return electron gyrofrequency in Hz
    """
    return echarge * B/emass/2/np.pi

In [ ]:
def f_va(B, n):
    """
    Return Alfven speed in SI unit.
    """
    return np.sqrt(B**2 / permeability/ n/ pmass)

##### Ex3
$n_e = n_p = 1cc$,
$B = 10nT$,
$\beta_{p\parallel}=0.1$

In [3]:
b_0 = 1e-8 # 10nT
n_p = 1e6 # 1cc
beta_p_par = 1

f_ce = echarge * b_0/emass/2/np.pi
va = np.sqrt(b_0**2 / permeability/ n_p/ pmass)
t_p_par = beta_p_par * b_0**2/(2 * permeability * n_p * boltzmann)
t_p_par_ev = t_p_par * boltzmann / echarge

print("f_ce = {0:.5g}".format(f_ce))
print("va = {0:.3g}".format(va)) 
print("t_p_par = {0:.3g}".format(t_p_par))
print("t_p_par_ev = {0:.5g}".format(t_p_par_ev))
print("aol = {0:.4g}".format(cspeed/va))

f_ce = 279.92
va = 2.18e+05
t_p_par = 2.88e+06
t_p_par_ev = 248.34
aol = 1374


In [ ]:
kz = 0.001
kz/np.sqrt(beta_p_par)

In [2]:
from py_vlasov.parallel_mode import parallel_em_wave_wrapper

In [122]:
k=0.1
betap = .1
t_list=[1.,1.]
a_list=[0,0.]
n_list=[1.,1.] 
q_list=[1.,-1.]
m_list=[1., emass/pmass]
v_list=[0,0]

In [123]:
wrel = 0
res_r = parallel_em_wave_wrapper(wrel, k, betap, t_list, \
                           a_list, n_list, q_list, \
                           m_list, v_list, method = 'pade', aol=1/5000, mode = 'r')
res_l = parallel_em_wave_wrapper(wrel, k, betap, t_list, \
                           a_list, n_list, q_list, \
                           m_list, v_list, method = 'numpy', aol=1/5000, mode = 'r')

In [124]:
f = lambda wrel: real_imag(parallel_em_wave_wrapper(wrel[0] + 1j * wrel[1], k, betap, t_list, \
                                                      a_list, n_list, q_list, \
                                                      m_list, v_list, method = 'pade', aol=1/5000))
#guess = k * np.sqrt(abs(1/betap - a_list[0]/2 - t_list[1] * a_list[1]/2)) 
#print('guess = ', guess)
guess = 0.1
freq = scipy.optimize.fsolve(f, real_imag(guess))
print(freq)

[  3.71544712e-01  -3.87055301e-09]
